In [55]:
import urllib.request
import os
import tarfile

In [56]:
url ="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath ="data/aclImdb_v1.tar.gz"
if not os.path.isfile(filepath):
    result=urllib.request.urlretrieve(url,filepath)
    print('downloaded:',result)

In [57]:
#解压缩文件
if not os.path.exists("data/aclImdb"):
    tfile = tarfile.open("data/aclImdb_v1.tar.gz",'r:gz')
    result=tfile.extractall('data/')

In [58]:
from keras.preprocessing.text import Tokenizer  #建立字典
from keras.preprocessing import sequence        #将用于截长补短让所有“数字列表”长度为100



In [59]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('',text)

In [60]:
#创建read——file函数读取IMDb文件目录
import os 
def read_files(filetype):
    path='data/aclImdb'
    file_list=[]
    positive_path =path+'/'+filetype+"/pos/"
    for f in os.listdir(positive_path):
        file_list += [positive_path+f]
    
    negative_path =path +'/'+filetype+'/neg/'
    for f in os.listdir(negative_path):
        file_list += [negative_path+f]
    print('read',filetype,'files:',len(file_list))
    
    all_labels =([1]*12500+[0]*12500)
    
    all_texts=[]
    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:
            all_texts +=[rm_tags("".join(file_input.readlines()))]
    
    return all_labels,all_texts
       

In [61]:
y_train,train_text =read_files("train")

read train files: 25000


In [62]:
y_test,test_text=read_files("test")

read train files: 25000


In [63]:
#查看信息
train_text[0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [64]:
y_train[0]

1

In [65]:
#建立token
token =Tokenizer(num_words=2000)
token.fit_on_texts(train_text)  #对每个单词出现的次数进行一个排序，取前2000列入字典
#查看读取多少文章
print(token.document_count)

25000


In [66]:
#每个单词出现的次数的排名
print(token.word_index)

{"individual's": 18945, "lawrence's": 26895, 'foal': 55194, 'oysters': 88195, 'spotlight': 7580, 'outmatched': 50184, 'tactics': 6907, "also'": 45591, "'few'": 84171, 'told': 575, 'linger': 12828, 'barbaric': 15884, 'resignedly': 84925, 'albas': 87041, 'anathema': 38640, 'steadier': 48569, 'britains': 33721, 'deduction': 31053, 'flirtations': 31234, "deeds'": 81248, 'refreshes': 43281, 'waking': 7625, 'mcguffin': 26561, "tomaselli's": 72741, 'horrendousness': 88782, 'glaser': 24845, 'anaesthetic': 48881, 'cityscape': 24677, 'worshiping': 26470, 'propositions': 37152, 'barrat': 31088, 'tessering': 71873, 'stayover': 84931, 'saith': 61865, 'budding': 8191, 'expiation': 72491, 'bukowski': 28662, 'compositions': 9042, 'scuttle': 30143, 'fifty': 5256, 'rotoscope': 38758, 'checkout': 17720, 'yossarian': 60617, '61': 24731, 'shock': 1461, "moonlighting'": 87520, 'shrills': 69014, 'discoveries': 18260, 'tenaru': 66484, 'museums': 24288, 'spaceman': 77587, 'stunk': 10294, 'capita': 83947, 'pola

In [67]:
#将影评文字转换为数字列表
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [68]:
print(train_text[0])


Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


In [69]:
print(x_train_seq[0])

[308, 6, 3, 1068, 208, 8, 29, 1, 168, 54, 13, 45, 81, 40, 391, 109, 137, 13, 57, 149, 7, 1, 481, 68, 5, 260, 11, 6, 72, 5, 631, 70, 6, 1, 5, 1, 1530, 33, 66, 63, 204, 139, 64, 1229, 1, 4, 1, 222, 899, 28, 68, 4, 1, 9, 693, 2, 64, 1530, 50, 9, 215, 1, 386, 7, 59, 3, 1470, 798, 5, 176, 1, 391, 9, 1235, 29, 308, 3, 352, 343, 142, 129, 5, 27, 4, 125, 1470, 5, 308, 9, 532, 11, 107, 1466, 4, 57, 554, 100, 11, 308, 6, 226, 47, 3, 11, 8, 214]


In [70]:
#转化为数字列表长度相等
x_train =sequence.pad_sequences(x_train_seq,maxlen=100)
x_test =sequence.pad_sequences(x_test_seq,maxlen=100)


In [71]:
#大于100，从前面截取，只留100个
print('before pad_sequences length=',len(x_train_seq[0]))
print(x_train_seq[0])

before pad_sequences length= 106
[308, 6, 3, 1068, 208, 8, 29, 1, 168, 54, 13, 45, 81, 40, 391, 109, 137, 13, 57, 149, 7, 1, 481, 68, 5, 260, 11, 6, 72, 5, 631, 70, 6, 1, 5, 1, 1530, 33, 66, 63, 204, 139, 64, 1229, 1, 4, 1, 222, 899, 28, 68, 4, 1, 9, 693, 2, 64, 1530, 50, 9, 215, 1, 386, 7, 59, 3, 1470, 798, 5, 176, 1, 391, 9, 1235, 29, 308, 3, 352, 343, 142, 129, 5, 27, 4, 125, 1470, 5, 308, 9, 532, 11, 107, 1466, 4, 57, 554, 100, 11, 308, 6, 226, 47, 3, 11, 8, 214]


In [72]:
print('after pad_sequences length=',len(x_train[0]))
print(x_train[0])

after pad_sequences length= 100
[  29    1  168   54   13   45   81   40  391  109  137   13   57  149
    7    1  481   68    5  260   11    6   72    5  631   70    6    1
    5    1 1530   33   66   63  204  139   64 1229    1    4    1  222
  899   28   68    4    1    9  693    2   64 1530   50    9  215    1
  386    7   59    3 1470  798    5  176    1  391    9 1235   29  308
    3  352  343  142  129    5   27    4  125 1470    5  308    9  532
   11  107 1466    4   57  554  100   11  308    6  226   47    3   11
    8  214]


In [73]:
#小于100，从前面添加，直到100个
print('before pad_sequences length=',len(x_train_seq[6]))
print(x_train_seq[6])

before pad_sequences length= 88
[418, 90, 31, 494, 5, 93, 3, 547, 1779, 706, 1, 61, 7, 323, 133, 21, 88, 56, 1493, 8, 1444, 474, 235, 30, 1691, 1, 7, 1, 18, 66, 302, 1739, 2, 66, 238, 85, 72, 21, 353, 1, 18, 186, 1, 110, 6, 51, 1724, 1, 16, 148, 1639, 21, 2, 127, 21, 191, 5, 397, 21, 1531, 1, 459, 6, 48, 357, 4, 5, 4, 835, 2, 6, 48, 51, 323, 301, 54, 102, 44, 21, 22, 263, 5, 141, 2, 838, 3, 342, 61]


In [74]:
print('after pad_sequences length=',len(x_train[6]))
print(x_train[6])

after pad_sequences length= 100
[   0    0    0    0    0    0    0    0    0    0    0    0  418   90
   31  494    5   93    3  547 1779  706    1   61    7  323  133   21
   88   56 1493    8 1444  474  235   30 1691    1    7    1   18   66
  302 1739    2   66  238   85   72   21  353    1   18  186    1  110
    6   51 1724    1   16  148 1639   21    2  127   21  191    5  397
   21 1531    1  459    6   48  357    4    5    4  835    2    6   48
   51  323  301   54  102   44   21   22  263    5  141    2  838    3
  342   61]


In [81]:
#提供嵌入层可以将“数字列表”转换为“向量列表”
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding 
#建立线性堆叠模型
model =Sequential()
#将“嵌入层”加入模型
model.add(Embedding(output_dim=32,input_dim=2000,input_length=100))
#Dropout层以避免过度拟合，每次迭代会随机地在神经网络中放弃20%的神经元
model.add(Dropout(0.2))
#将“平坦层”加入模型
model.add(Flatten())
#将“隐藏层”加入模型
model.add(Dense(units=256,activation='relu'))  #隐藏层有256个神经元
model.add(Dropout(0.35))
#将
model.add(Dense(units=1,activation='sigmoid'))

In [82]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 32)           64000     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100, 32)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               819456    
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 257       
Total params: 883,713
Trainable params: 883,713
Non-trainable params: 0
_________________________________________________________________


In [83]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
train_history = model.fit(x_train,y_train,batch_size=100,epochs=10,verbose=2,validation_split=0.2)


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 6s - loss: 0.4751 - acc: 0.7586 - val_loss: 0.5176 - val_acc: 0.7618
Epoch 2/10
 - 5s - loss: 0.2677 - acc: 0.8902 - val_loss: 0.5191 - val_acc: 0.7776
Epoch 3/10
 - 5s - loss: 0.1610 - acc: 0.9418 - val_loss: 0.5654 - val_acc: 0.7894
Epoch 4/10
 - 6s - loss: 0.0864 - acc: 0.9705 - val_loss: 0.7735 - val_acc: 0.7680
Epoch 5/10
 - 5s - loss: 0.0506 - acc: 0.9827 - val_loss: 1.0958 - val_acc: 0.7314
Epoch 6/10
 - 5s - loss: 0.0356 - acc: 0.9882 - val_loss: 1.2045 - val_acc: 0.7384
Epoch 7/10
 - 5s - loss: 0.0288 - acc: 0.9896 - val_loss: 1.0442 - val_acc: 0.7844
Epoch 8/10
 - 5s - loss: 0.0284 - acc: 0.9895 - val_loss: 1.4167 - val_acc: 0.7296
Epoch 9/10
 - 5s - loss: 0.0250 - acc: 0.9910 - val_loss: 1.3344 - val_acc: 0.7528
Epoch 10/10
 - 5s - loss: 0.0271 - acc: 0.9902 - val_loss: 1.3357 - val_acc: 0.7450


In [84]:
scores=model.evaluate(x_test,y_test,verbose=1)
scores[1]

25000/25000 [==============================] - 1s 50us/step


0.94876

In [85]:
predict=model.predict_classes(x_test)
predict[:10]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [87]:
#将二维数组转化为一维数组
predict_classes=predict.reshape(-1)
predict_classes[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [92]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

model=Sequential()
model.add(Embedding(output_dim=32,input_dim=3800,input_length=380))
model.add(Dropout(0.35))

model.add(SimpleRNN(units=16))

model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.35))

model.add(Dense(units=1,activation='sigmoid'))

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_17 (Dropout)         (None, 380, 32)           0         
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense_17 (Dense)             (None, 256)               4352      
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 257       
Total params: 126,993
Trainable params: 126,993
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
train_history = model.fit(x_train,y_train,batch_size=380,epochs=10,verbose=2,validation_split=0.2)


ValueError: Error when checking input: expected embedding_9_input to have shape (380,) but got array with shape (100,)

In [89]:
scores=model.evaluate(x_test,y_test,verbose=1)
scores[1]

RuntimeError: The model needs to be compiled before being used.